In [1]:
import os
from utils import utils
import pandas as pd
import torch
from datetime import datetime
from collections import Counter
from sklearn.metrics import f1_score, recall_score, precision_score
import numpy as np
from transformers.trainer_utils import set_seed
from transformers import RobertaTokenizer
from transformers import AutoConfig, AutoModelWithHeads
from transformers.adapters.composition import Stack
from transformers import TrainingArguments, EvalPrediction, AdapterTrainer
from utils.evaluation import compute_pearsonr


dateTimeObj = datetime.now()

RANDOM_SEED = 42
set_seed(RANDOM_SEED)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Data

In [2]:
""" load data """

train_data, val_data, test_data = utils.load_wassa_dataset()

In [3]:
""" get task labels """

prediction_task = 'empathy'

train_labels = list(train_data[prediction_task].values)
val_labels = list(val_data[prediction_task].values)
test_labels = list(test_data[prediction_task].values)

In [4]:
""" Prepare dataset for training: feature encodings """

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

train_encodings = tokenizer(list(train_data['essay'].values), truncation=True, padding=True)
val_encodings = tokenizer(list(val_data['essay'].values), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data['essay'].values), truncation=True, padding=True)

In [5]:
""" setup torch dataset """

class WassaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = WassaDataset(train_encodings, train_labels)
val_dataset = WassaDataset(val_encodings, val_labels)
test_dataset = WassaDataset(test_encodings, test_labels)

# Stacking Emotion Prediction

In [6]:
""" init model """

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1,
    hidden_dropout_prob=.01,
)
model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

/home/nbalguri/.local/lib/python3.9/site-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/home/nbalguri/.local/lib/python3.9/site-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertF

In [7]:
""" load emotion adapter, using: https://huggingface.co/AdapterHub/roberta-base-pf-emotion """

emotion_adapter = model.load_adapter('AdapterHub/roberta-base-pf-emotion', source="hf")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
""" add adapter for emotion prediction task """

adapter_name = "EMP_emotion_stack" if prediction_task == 'empathy' else 'DIS_emotion_stack'

model.add_adapter(adapter_name)
model.add_classification_head(adapter_name, num_labels=1)

In [9]:
""" activate adapter stack """

model.active_adapters = Stack(emotion_adapter, adapter_name)

In [10]:
""" activate adapter for training """

model.train_adapter([adapter_name])

In [11]:
""" training arguments """

approach = 'emotion-stack'
training_output_dir = f"./training_output/{approach}/{prediction_task}_{dateTimeObj.hour}{dateTimeObj.minute}-{dateTimeObj.day}-{dateTimeObj.month}"
num_train_epochs=20
per_device_train_batch_size=8
per_device_eval_batch_size=8
metric_for_best_model='eval_pearsonr'
warmup_steps=1000
weight_decay=0.1
learning_rate=1e-04


training_args = TrainingArguments(
    seed=RANDOM_SEED,
    output_dir=training_output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    learning_rate=learning_rate,
     warmup_steps=warmup_steps,
     weight_decay=weight_decay,
    logging_dir='./logs',
    logging_steps=50,
    eval_steps=50,
    save_steps=50,
    evaluation_strategy='steps',
    disable_tqdm=False,
    overwrite_output_dir=True,
    remove_unused_columns=False,
    save_strategy='steps',
    load_best_model_at_end=True,
    metric_for_best_model=metric_for_best_model,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
""" setup trainer """

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_pearsonr,
)

In [ ]:
""" train """

trainer.train()

/home/nbalguri/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1860
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4660
  Number of trainable parameters = 2081095


Step,Training Loss,Validation Loss,Pearsonr,Pearsonr Scipy,Pval
50,19.310400,16.894705,0.034300,0.034259,0.575144
100,16.912900,14.828650,-0.023100,-0.023074,0.705845
150,12.510200,9.373271,-0.039700,-0.039704,0.515929
200,5.501000,3.529942,0.040600,0.040616,0.506325
250,4.256400,3.434055,0.116400,0.116411,0.056074
300,3.684500,3.318197,0.223800,0.223845,0.000209
350,3.132900,3.241638,0.300100,0.300106,0.000001
400,3.234400,3.046634,0.357000,0.356980,0.000000
450,2.968300,2.955862,0.380900,0.380938,0.000000
500,2.655200,3.408524,0.403600,0.403595,0.000000


***** Running Evaluation *****
  Num examples = 270
  Batch size = 8
Saving model checkpoint to ./training_output/emotion-stack/empathy_154-7-4/checkpoint-50
Configuration saved in ./training_output/emotion-stack/empathy_154-7-4/checkpoint-50/emotion/adapter_config.json
Module weights saved in ./training_output/emotion-stack/empathy_154-7-4/checkpoint-50/emotion/pytorch_adapter.bin
Configuration saved in ./training_output/emotion-stack/empathy_154-7-4/checkpoint-50/emotion/head_config.json
Module weights saved in ./training_output/emotion-stack/empathy_154-7-4/checkpoint-50/emotion/pytorch_model_head.bin
Configuration saved in ./training_output/emotion-stack/empathy_154-7-4/checkpoint-50/EMP_emotion_stack/adapter_config.json
Module weights saved in ./training_output/emotion-stack/empathy_154-7-4/checkpoint-50/EMP_emotion_stack/pytorch_adapter.bin
Configuration saved in ./training_output/emotion-stack/empathy_154-7-4/checkpoint-50/EMP_emotion_stack/head_config.json
Module weights saved 

# Evaluate

In [ ]:
""" output eval metrics from best model """

#trainer.model.cuda()
eval_output = trainer.evaluate()
eval_result = eval_output[metric_for_best_model]

pd.DataFrame({'metric':list(eval_output.keys()), 'value': list(eval_output.values())}, columns=['metric', 'value'])

# Test predictions

In [ ]:
""" make predictions on test dataset """

p = trainer.predict(test_dataset)
preds = p.predictions[:, 0]


In [ ]:
""" save predictions """

pred_file = f"{approach}-{metric_for_best_model}-{round(eval_result * 100, 4)}_{dateTimeObj.hour}{dateTimeObj.minute}-{dateTimeObj.day}-{dateTimeObj.month}.tsv"
pred_path = f'./predictions/{prediction_task}/{pred_file}'
os.makedirs(f'./predictions/{prediction_task}/', exist_ok=True)
pd.Series(preds).to_csv(pred_path, sep='\t', header=False, index=False)
print("saved predictions to",pred_path)

# Save the adapter

In [17]:
# trainer.model.save_adapter(f"./trained_adapters/{adapter_name}", adapter_name)